# New methane lifetime treatment within FaIR

Use the option `ch4_method = thornhill2021`.

We should try this first with concentrations and then with emissions, both with prescribed temperatures.

Then in another notebook, we will try a full emissions-driven run with freely evolving temperatures.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
samples = 1

In [ ]:
# we focus on a stylised SSP3-7.0 pathway has has historical warming to 2020 and a straight line up to 4C at 2100.
# note that this will not be correct for other SSPs, but temperature feedback effect on methane lifetime is
# relatively small.

df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(351)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)

In [ ]:
species = ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'NOx', 'VOC',
          'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b',
          'HCFC-142b', 'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
          'Halon-1211', 'Halon-1301', 'Halon-2402']

In [ ]:
species, properties = read_properties(species=species)

In [ ]:
properties

In [ ]:
scenarios_to_include = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# for benchmarking FaIR calculated concentrations
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
conc_ch4 = {}
for scenario in scenarios_to_include:
    conc_ch4[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
        (df_conc['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
conc_ch4

In [ ]:
f=FAIR()
f.define_time(1750, 2100, 1)
f.define_scenarios(scenarios_to_include)
f.define_species(species, properties)
f.define_configs(list(range(samples)))
f.temperature_prescribed=True
f.ch4_method='thornhill2021'
f.ghg_method='leach2021'

In [ ]:
f.allocate()

In [ ]:
fill(f.temperature, gmst[:, None, None], layer=0)

In [ ]:
f.fill_from_rcmip()
f.fill_species_configs()

In [ ]:
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

In [ ]:
initialise(f.forcing, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.concentration, f.species_configs['baseline_concentration'])

In [ ]:
f.run()

In [ ]:
f.concentration.loc[dict(scenario='ssp370', specie='CH4')]

In [ ]:
pl.plot(f.timebounds, f.concentration.loc[dict(specie='CH4', config=0)], label=scenarios_to_include)
pl.plot(f.timebounds, conc_ch4['ssp370'][:351], color='k', label='CMIP6 hist+SSP3-7.0')
pl.legend()

In [ ]:
pl.plot(f.alpha_lifetime[:,3,:,0]);

In [ ]:
pl.plot(f.alpha_lifetime[:,3,:,0] * f.species_configs['unperturbed_lifetime'][0,0,0]);

In [ ]:
pl.plot(f.temperature[:,3,:,0]);